# Neptune Result Fetching Notebook

In [21]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3

# required to enable non-blocking interaction:
%gui qt5

# import ipywidgets as widgets
# widgets.Widget.control_comm_live = False

import sys
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from neuropy.utils.result_context import IdentifyingContext
from nptyping import NDArray
import neuropy.utils.type_aliases as types
from collections import defaultdict

import numpy as np
import pandas as pd
import re
from pathlib import Path
from datetime import datetime

from neuropy.utils.indexing_helpers import PandasHelpers
from pyphocorehelpers.indexing_helpers import partition_df
# Set the maximum number of columns to display
pd.set_option('display.max_columns', 100)

import IPython
from pyphocorehelpers.programming_helpers import IPythonHelpers
from pyphocorehelpers.notebook_helpers import NotebookCellExecutionLogger
from pyphocorehelpers.assertion_helpers import Assert

# Jupyter-lab enable printing for any line on its own (instead of just the last one in the cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# _notebook_path:Path = Path(IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())).resolve() # Finds the path of THIS notebook

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')

# Switch to the desired interactivity mode
plt.interactive(True)

import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
template: str = 'plotly_dark' # set plotl template
pio.templates.default = template
from pyphocorehelpers.plotting.media_output_helpers import fig_to_clipboard
from pyphocorehelpers.Filesystem.path_helpers import file_uri_from_path, sanitize_filename_for_Windows
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, simple_path_display_widget
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_helper_save_figures, _helper_build_figure, plotly_pre_post_delta_scatter, plot_across_sessions_scatter_results
from pyphocorehelpers.assertion_helpers import Assert

# from ..PendingNotebookCode import plot_across_sessions_scatter_results, plot_histograms, plot_stacked_histograms
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import find_csv_files, find_HDF5_files, find_most_recent_files
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import plot_histograms_across_sessions, plot_histograms, plot_stacked_histograms

from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DecoderDecodedEpochsResult
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import load_across_sessions_exported_files, _process_and_load_exported_file, _common_cleanup_operations

from pyphocorehelpers.programming_helpers import metadata_attributes
from pyphocorehelpers.function_helpers import function_attributes

from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import build_session_t_delta, _new_process_csv_files, _old_process_csv_files

debug_print: bool = False

_TODAY_DAY_ONLY_DATE: str = "2024-09-27"
TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_Apogee"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_GL"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_Lab"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_rMBP"

print(f'TODAY_DAY_DATE: {TODAY_DAY_DATE}')

types.session_str: TypeAlias = str # a unique session identifier

import neptune # for logging progress and results
from neptune.types import File
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import Neptuner, AutoValueConvertingNeptuneRun, set_environment_variables, SessionDescriptorString, RunID, NeptuneRunCollectedResults
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import KnownNeptuneProjects

## Gets the notebook filepath for Neptune:
import IPython
from pyphocorehelpers.programming_helpers import IPythonHelpers
# notebook_filepath: str = IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())
# assert Path(notebook_filepath).resolve().exists(), f"found notebook filepath: '{notebook_filepath}' does not exist"
# notebook_filepath
        
# neptune_kwargs = KnownNeptuneProjects.get_PhoDibaBatchProcessing_neptune_kwargs()
neptune_kwargs = KnownNeptuneProjects.get_PhoDibaLongShortUpdated_neptune_kwargs()
neptuner = Neptuner(project_name=neptune_kwargs['project'], api_token=neptune_kwargs['api_token'])
# neptuner
# neptuner.project
# neptune_root_output_path = Path('EXTERNAL/PhoDibaPaper2024Book/data/neptune').resolve()
neptune_root_output_path = Path(r"C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\PhoDibaPaper2024Book\data\neptune").resolve()
# neptune_root_output_path = Path("/home/halechr/repos/Spike3D/EXTERNAL/PhoDibaPaper2024Book/data").resolve()
neptune_root_output_path.mkdir(exist_ok=True)
neptune_root_output_path
# neptune_project_output_path = neptune_root_output_path.joinpath(neptuner.project_name).resolve()
neptune_project_output_path = neptune_root_output_path.joinpath(neptuner.project_name).resolve()
neptune_project_output_path.mkdir(exist_ok=True, parents=True)
neptune_project_output_path

neptune_project_figures_output_path = neptune_project_output_path.joinpath('figs').resolve()
neptune_project_figures_output_path.mkdir(exist_ok=True, parents=True)
neptune_project_figures_output_path

neptune_logs_output_path = neptune_project_output_path.joinpath('logs').resolve()
neptune_logs_output_path.mkdir(exist_ok=True, parents=True)
neptune_logs_output_path

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
TODAY_DAY_DATE: 2024-09-27_Apogee
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/


WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/neptune')

WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/neptune/commander.pho/PhoDibaLongShortUpdated')

WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/neptune/commander.pho/PhoDibaLongShortUpdated/figs')

WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/neptune/commander.pho/PhoDibaLongShortUpdated/logs')

In [2]:
neptune_run_collected_results: NeptuneRunCollectedResults = neptuner.get_most_recent_session_runs(oldest_included_run_date='2024-09-24', n_recent_results=3)
# neptune_run_collected_results: NeptuneRunCollectedResults = neptuner.get_most_recent_session_runs(oldest_included_run_date='2024-09-20', n_recent_results=3)
# neptune_run_collected_results: NeptuneRunCollectedResults = neptuner.get_most_recent_session_runs(oldest_included_run_date='2024-09-20', n_recent_results=3)
neptune_run_collected_results
# runs_dict, most_recent_runs_table_df = neptuner.get_most_recent_session_runs(oldest_included_run_date='2024-06-01', n_recent_results=2)
# most_recent_runs_table_df

Fetching table...: 0 [00:00, ?/s]

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/e/LS2023-1371
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/e/LS2023-1370
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/e/LS2023-1369
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/e/LS2023-1368
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/e/LS2023-1367
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/e/LS2023-1366
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/e/LS2023-1365
[neptune] [info   ] Neptune initialized. Open in the app: https://app

In [3]:
context_indexed_runs_list_dict: Dict[IdentifyingContext, List[AutoValueConvertingNeptuneRun]] = neptune_run_collected_results.context_indexed_runs_list_dict
# context_indexed_runs_list_dict

# most_recent_runs_table_df
context_indexed_run_logs: Dict[IdentifyingContext, str] = neptune_run_collected_results.context_indexed_run_logs
# context_indexed_run_logs

# run_logs: Dict[SessionDescriptorString, str] = neptune_run_collected_results.run_logs
# run_logs

## INPUTS: neptuner, run_logs, most_recent_runs_table_df

# session_column_individual_variables = ['format_name', 'animal', 'exper_name', 'session_name']
# session_column_variables = ['session_descriptor_string']
# processing_status_column_names = ['sys/id', 'sys/hostname', 'sys/creation_time', 'sys/running_time', 'sys/ping_time', 'sys/monitoring_time', 'sys/size', 'sys/tags', 'source_code/entrypoint']
# processing_status_column_names = ['sys/id', 'sys/hostname', 'sys/creation_time', 'sys/running_time', 'sys/ping_time', 'sys/monitoring_time', 'sys/size', 'sys/tags', 'source_code/entrypoint']
# processing_status_column_names = ['id', 'hostname', 'creation_time', 'running_time', 'ping_time', 'monitoring_time', 'size', 'tags', 'entrypoint']

most_recent_runs_table_df: pd.DataFrame = neptune_run_collected_results.most_recent_runs_table_df
most_recent_runs_table_df
most_recent_runs_session_descriptor_string_to_context_map: Dict[SessionDescriptorString, IdentifyingContext] = neptune_run_collected_results.most_recent_runs_session_descriptor_string_to_context_map
## INPUTS: most_recent_runs_session_descriptor_string_to_context_map, run_logs
context_indexed_run_logs: Dict[IdentifyingContext, str] = neptune_run_collected_results.context_indexed_run_logs # get the IdentifyingContext indexed item
# context_indexed_run_logs

# 'outputs/log'

## INPUTS: most_recent_runs_table_df
most_recent_runs_context_indexed_run_extra_data: Dict[IdentifyingContext, Dict] = neptune_run_collected_results.most_recent_runs_context_indexed_run_extra_data
# most_recent_runs_context_indexed_run_extra_data # SessionTuple(format_name='kdiba', animal='pin01', exper_name='one', session_name='11-02_17-46-44', session_descriptor_string='kdiba_pin01_one_11-02_17-46-44_sess', id='LS2023-1335', hostname='gl3126.arc-ts.umich.edu', creation_time=Timestamp('2024-08-29 16:39:16.613000'), running_time=8735.629, ping_time=Timestamp('2024-09-24 08:38:06.626000'), monitoring_time=1543, size=28686905.0, tags='11-02_17-46-44,one,kdiba,pin01', entrypoint='figures_kdiba_pin01_one_11-02_17-46-44.py')

# ['script_type']

## OUTPUTS: most_recent_runs_session_descriptor_string_to_context_map, context_indexed_run_logs, most_recent_runs_context_indexed_run_extra_data

,creation_time,description,failed,group_tags,hostname,id,modification_time,monitoring_time,name,owner,ping_time,running_time,size,state,tags,trashed,animal,exper_name,format_name,session_descriptor_string,session_name,entrypoint,git
0,2024-09-27 01:53:50.822,,False,,gl3151.arc-ts.umich.edu,LS2023-1371,2024-09-27 01:54:12.213,18,Untitled,commander.pho,2024-09-27 01:56:39.355,168.525,35866309.0,Inactive,"gor01,two,2006-6-08_21-16-25,kdiba",False,gor01,two,kdiba,kdiba_gor01_two_2006-6-08_21-16-25_sess,2006-6-08_21-16-25,NaN,9dd361af7023578e85667f33bff9958f73877f49
1,2024-09-26 23:42:29.831,,False,,gl3151.arc-ts.umich.edu,LS2023-1370,2024-09-27 01:53:11.610,30,Untitled,commander.pho,2024-09-27 01:53:11.610,3552.545,79622445.0,Inactive,"gor01,2006-6-07_16-40-19,two,kdiba",False,gor01,two,kdiba,kdiba_gor01_two_2006-6-07_16-40-19_sess,2006-6-07_16-40-19,NaN,9dd361af7023578e85667f33bff9958f73877f49
2,2024-09-26 23:38:42.638,,False,,gl3151.arc-ts.umich.edu,LS2023-1369,2024-09-26 23:41:30.996,163,Untitled,commander.pho,2024-09-26 23:41:30.996,168.351,53852089.0,Inactive,"gor01,one,2006-6-12_15-55-31,kdiba",False,gor01,one,kdiba,kdiba_gor01_one_2006-6-12_15-55-31_sess,2006-6-12_15-55-31,NaN,9dd361af7023578e85667f33bff9958f73877f49
3,2024-09-26 22:15:22.256,,False,,gl3151.arc-ts.umich.edu,LS2023-1368,2024-09-26 23:37:58.759,52,Untitled,commander.pho,2024-09-26 23:37:58.759,2420.797,97688467.0,Inactive,"gor01,one,2006-6-09_1-22-43,kdiba",False,gor01,one,kdiba,kdiba_gor01_one_2006-6-09_1-22-43_sess,2006-6-09_1-22-43,NaN,9dd361af7023578e85667f33bff9958f73877f49
4,2024-09-26 20:04:12.728,,False,,gl3151.arc-ts.umich.edu,LS2023-1367,2024-09-26 22:14:36.671,30,Untitled,commander.pho,2024-09-26 22:14:36.671,3276.621,64017238.0,Inactive,"gor01,one,kdiba,2006-6-08_14-26-15",False,gor01,one,kdiba,kdiba_gor01_one_2006-6-08_14-26-15_sess,2006-6-08_14-26-15,NaN,9dd361af7023578e85667f33bff9958f73877f49
5,2024-09-26 19:54:03.665,,False,,gl3151.arc-ts.umich.edu,LS2023-1366,2024-09-26 20:03:53.204,590,Untitled,commander.pho,2024-09-26 20:03:53.204,589.497,55133.0,Inactive,"2006-6-09_22-24-40,gor01,two,kdiba",False,gor01,two,kdiba,kdiba_gor01_two_2006-6-09_22-24-40_sess,2006-6-09_22-24-40,figures_kdiba_gor01_two_2006-6-09_22-24-40.py,NaN
6,2024-09-26 18:07:18.635,,False,,RDLU0039,LS2023-1365,2024-09-26 18:07:25.787,6,Untitled,commander.pho,2024-09-26 19:14:57.618,2082.457,125206.0,Inactive,"one,gor01,2006-6-08_14-26-15,kdiba",False,gor01,one,kdiba,kdiba_gor01_one_2006-6-08_14-26-15_sess,2006-6-08_14-26-15,NaN,33ac89fbe949834c027943c9a4eb54887f208457
7,2024-09-24 22:33:44.029,,False,,gl3062.arc-ts.umich.edu,LS2023-1361,2024-09-25 00:50:53.585,8230,Untitled,commander.pho,2024-09-25 00:50:53.585,3688.118,64141629.0,Inactive,"one,gor01,kdiba,2006-6-08_14-26-15",False,gor01,one,kdiba,kdiba_gor01_one_2006-6-08_14-26-15_sess,2006-6-08_14-26-15,figures_kdiba_gor01_one_2006-6-08_14-26-15.py,NaN
8,2024-09-24 22:33:10.662,,False,,gl3067.arc-ts.umich.edu,LS2023-1354,2024-09-25 00:37:37.273,7467,Untitled,commander.pho,2024-09-25 00:37:37.273,3364.046,45650258.0,Inactive,"2006-6-07_16-40-19,two,kdiba,gor01",False,gor01,two,kdiba,kdiba_gor01_two_2006-6-07_16-40-19_sess,2006-6-07_16-40-19,figures_kdiba_gor01_two_2006-6-07_16-40-19.py,NaN
9,2024-09-24 22:32:55.056,,False,,gl3026.arc-ts.umich.edu,LS2023-1351,2024-09-25 00:32:43.297,7188,Untitled,commander.pho,2024-09-25 00:32:43.297,3009.733,31825763.0,Inactive,"kdiba,gor01,2006-6-07_11-26-53,one",False,gor01,one,kdiba,kdiba_gor01_one_2006-6-07_11-26-53_sess,2006-6-07_11-26-53,figures_kdiba_gor01_one_2006-6-07_11-26-53.py,NaN


In [54]:
"outputs/figures/display_fn_name:display_short_long_pf1D_comparison/track:long"
"outputs/figures/display_fn_name:display_short_long_pf1D_comparison/track:short"






# Particular Run: `https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/e/LS2023-1343`
# most_recent_runs_context_indexed_run_extra_data
a_session_descriptor_str: str = 'kdiba_gor01_two_2006-6-07_16-40-19'
a_run = neptune_run_collected_results.runs_dict['LS2023-1343'] # kdiba_gor01_two_2006-6-07_16-40-19_sess

# fig_input_key: str = "outputs/figures/display_fn_name:display_short_long_pf1D_comparison/track:long"
# fig_input_key: str = "display_fn_name:display_short_long_pf1D_comparison/track:long"
# fig_input_key: str = "display_fn_name:display_short_long_pf1D_comparison/track:short"
# download_image(a_run=a_run, fig_input_key=fig_input_key, a_session_descriptor_str=a_session_descriptor_str)


# Support Incomplete Keys to export multiple images: _________________________________________________________________ #
# fig_input_key: str = "display_fn_name:BatchPhoJonathanReplayFRC/plot_result_set:shared/page:1of6/aclus:(2,4,5,6,7,8,9,11,12,13)"
fig_input_key: str = "display_fn_name:BatchPhoJonathanReplayFRC" # plot_result_set:shared/page:1of6/aclus:(2,4,5,6,7,8,9,11,12,13)
# check if the key ends with a forward-slash, in which case it may describe multiple images
# fig_input_key.endswith('/')



'outputs/figures/display_fn_name:display_short_long_pf1D_comparison/track:long'

'outputs/figures/display_fn_name:display_short_long_pf1D_comparison/track:short'

In [59]:
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import get_nested_value
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import flatten_dict
# from neptune.types import File
from neptune.attributes.atoms.file import File

a_parsed_structure = a_run.get_structure().get('outputs', {}).get('figures', None)
if a_parsed_structure is None:
    raise ValueError(f'No "outputs/values" in this run.') #skip this one

# a_parsed_structure = a_run.get_structure()['outputs']['figures']
assert isinstance(a_parsed_structure, dict), f"type(a_parsed_structure): {type(a_parsed_structure)} instead of dict. a_parsed_structure: {a_parsed_structure}"
if debug_print:
    print(f'a_parsed_structure: {a_parsed_structure}')

# for k, v in a_parsed_structure.items():
#     # Flatten each nested dictionary and update the flattened_dict
#     _parsed_run_structure_dict[a_ctxt].update(flatten_dict(v, parent_key=k))

## parse the key:
fig_input_key_parts: List[str] = fig_input_key.split('/') # ['display_fn_name:display_short_long_pf1D_comparison', 'track:long']
fig_input_key_parts

## parse key:value pairs into split arrays
# [k.split(':') for k in fig_input_key_parts] # [['display_fn_name', 'display_short_long_pf1D_comparison'], ['track', 'short']]

fig_split_key_value_pair_parts = [k.split(':')[-1] for k in fig_input_key_parts] ## remove the keys from the path, ['display_short_long_pf1D_comparison', 'short']
# fig_split_key_value_pair_parts

## Find the figure or figure(s):

# a_fig_file_field = a_parsed_structure['display_fn_name:display_short_long_pf1D_comparison']["track:long"]
a_fig_file_field = get_nested_value(a_parsed_structure, fig_input_key_parts)
# a_fig_file_field

# {'plot_result_set:long_only': {'aclus:(48)': <neptune.attributes.atoms.file.File at 0x282be50c760>},
#  'plot_result_set:shared': {'page:1of3': {'aclus:(7,41,20,23,32,63,46,52,15,54,51,60,50,28,31,49,26,5,40,62)': <neptune.attributes.atoms.file.File at 0x282be50c7c0>},
#   'page:1of6': {'aclus:(2,4,5,6,7,8,9,11,12,13)': <neptune.attributes.atoms.file.File at 0x282be50caf0>},
#   'page:2of3': {'aclus:(11,55,27,45,24,53,17,56,6,35,34,8,25,21,39,33,44,64,43,2)': <neptune.attributes.atoms.file.File at 0x282be50cb20>},
#   'page:2of6': {'aclus:(14,15,16,17,20,21,22,23,24,25)': <neptune.attributes.atoms.file.File at 0x282be4f2040>},
#   'page:3of3': {'aclus:(4,9,12,13,14,16,22,29,30,36,37,38,42,47,57,58,59,61)': <neptune.attributes.atoms.file.File at 0x282be4df9d0>},
#   'page:3of6': {'aclus:(26,27,28,29,30,31,32,33,34,35)': <neptune.attributes.atoms.file.File at 0x282be4dfb80>},
#   'page:4of6': {'aclus:(36,37,38,39,40,41,42,43,44,45)': <neptune.attributes.atoms.file.File at 0x282be4dfc70>},
#   'page:5of6': {'aclus:(46,47,49,50,51,52,53,54,55,56)': <neptune.attributes.atoms.file.File at 0x282be4df8e0>},
#   'page:6of6': {'aclus:(57,58,59,60,61,62,63,64)': <neptune.attributes.atoms.file.File at 0x282be4df5b0>}},
#  'plot_result_set:short_only': {'aclus:(3,10,18,19,65)': <neptune.attributes.atoms.file.File at 0x282be4df760>}}

## Build the output file path
a_session_figures_output_path = neptune_project_figures_output_path.joinpath(a_session_descriptor_str)
a_session_figures_output_path.mkdir(exist_ok=True)

if not isinstance(a_fig_file_field, File):
    print(f'not a figure file! a dictionary instead probably: type(a_fig_file_field): {type(a_fig_file_field)}')
    a_flattened_figure_dict = flatten_dict(a_fig_file_field)
    for a_sub_key, a_sub_fig_file_field in a_flattened_figure_dict.items():
        ## Each sub-item:
        sub_fig_full_input_key: str = '/'.join([fig_input_key, a_sub_key])
        if debug_print:
            print(f'sub_fig_full_input_key: "{sub_fig_full_input_key}"')
        sub_fig_input_key_parts: List[str] = a_sub_key.split('/') # ['display_fn_name:display_short_long_pf1D_comparison', 'track:long']
        sub_fig_split_key_value_pair_parts = [k.split(':')[-1] for k in sub_fig_input_key_parts]
        a_fig_output_name: str = '-'.join(fig_split_key_value_pair_parts + sub_fig_split_key_value_pair_parts) + '.png'
        a_fig_output_path = a_session_figures_output_path.joinpath(a_fig_output_name).resolve()
        if debug_print:
            print(f'a_fig_output_path: "{a_fig_output_path}"')
        ## Try the download
        try:
            _a_download_result = a_sub_fig_file_field.download(destination=a_fig_output_path.as_posix())
            _context_fig_files_dict[a_session_descriptor_str][sub_fig_full_input_key] = a_fig_output_path.as_posix()
            if debug_print:
                print(f'\tdone.')
        except MissingFieldException as err:
            # print(f'MissingFieldException for a_run.id: {a_run_id} (err: {err})')
            print(f'MissingFieldException for a_run.id: {self}')
            pass
    
# {'plot_result_set:long_only/aclus:(48)': <neptune.attributes.atoms.file.File at 0x282be50c760>,
#  'plot_result_set:shared/page:1of3/aclus:(7,41,20,23,32,63,46,52,15,54,51,60,50,28,31,49,26,5,40,62)': <neptune.attributes.atoms.file.File at 0x282be50c7c0>,
#  'plot_result_set:shared/page:1of6/aclus:(2,4,5,6,7,8,9,11,12,13)': <neptune.attributes.atoms.file.File at 0x282be50caf0>,
#  'plot_result_set:shared/page:2of3/aclus:(11,55,27,45,24,53,17,56,6,35,34,8,25,21,39,33,44,64,43,2)': <neptune.attributes.atoms.file.File at 0x282be50cb20>,
#  'plot_result_set:shared/page:2of6/aclus:(14,15,16,17,20,21,22,23,24,25)': <neptune.attributes.atoms.file.File at 0x282be4f2040>,
#  'plot_result_set:shared/page:3of3/aclus:(4,9,12,13,14,16,22,29,30,36,37,38,42,47,57,58,59,61)': <neptune.attributes.atoms.file.File at 0x282be4df9d0>,
#  'plot_result_set:shared/page:3of6/aclus:(26,27,28,29,30,31,32,33,34,35)': <neptune.attributes.atoms.file.File at 0x282be4dfb80>,
#  'plot_result_set:shared/page:4of6/aclus:(36,37,38,39,40,41,42,43,44,45)': <neptune.attributes.atoms.file.File at 0x282be4dfc70>,
#  'plot_result_set:shared/page:5of6/aclus:(46,47,49,50,51,52,53,54,55,56)': <neptune.attributes.atoms.file.File at 0x282be4df8e0>,
#  'plot_result_set:shared/page:6of6/aclus:(57,58,59,60,61,62,63,64)': <neptune.attributes.atoms.file.File at 0x282be4df5b0>,
#  'plot_result_set:short_only/aclus:(3,10,18,19,65)': <neptune.attributes.atoms.file.File at 0x282be4df760>}


['display_fn_name:BatchPhoJonathanReplayFRC']

not a figure file! a dictionary instead probably: type(a_fig_file_field): <class 'dict'>


{'plot_result_set:long_only/aclus:(48)': <neptune.attributes.atoms.file.File at 0x282be50c760>,
 'plot_result_set:shared/page:1of3/aclus:(7,41,20,23,32,63,46,52,15,54,51,60,50,28,31,49,26,5,40,62)': <neptune.attributes.atoms.file.File at 0x282be50c7c0>,
 'plot_result_set:shared/page:1of6/aclus:(2,4,5,6,7,8,9,11,12,13)': <neptune.attributes.atoms.file.File at 0x282be50caf0>,
 'plot_result_set:shared/page:2of3/aclus:(11,55,27,45,24,53,17,56,6,35,34,8,25,21,39,33,44,64,43,2)': <neptune.attributes.atoms.file.File at 0x282be50cb20>,
 'plot_result_set:shared/page:2of6/aclus:(14,15,16,17,20,21,22,23,24,25)': <neptune.attributes.atoms.file.File at 0x282be4f2040>,
 'plot_result_set:shared/page:3of3/aclus:(4,9,12,13,14,16,22,29,30,36,37,38,42,47,57,58,59,61)': <neptune.attributes.atoms.file.File at 0x282be4df9d0>,
 'plot_result_set:shared/page:3of6/aclus:(26,27,28,29,30,31,32,33,34,35)': <neptune.attributes.atoms.file.File at 0x282be4dfb80>,
 'plot_result_set:shared/page:4of6/aclus:(36,37,38,39,

In [ ]:
_context_log_files_dict = neptune_run_collected_results.download_uploaded_log_files(neptune_logs_output_path=neptune_logs_output_path)
_context_log_files_dict

##  2024-09-24 - tries to get the figures from the runs

In [61]:
# fig_input_key: str = "display_fn_name:display_long_short_laps"
# fig_input_key: str = "display_fn_name:display_short_long_pf1D_comparison/track:long"
# fig_input_key: str = "display_fn_name:display_short_long_pf1D_comparison/track:short"
# fig_input_key: str = "display_fn_name:running_and_replay_speeds_over_time"

# Support Incomplete Keys to export multiple images: _________________________________________________________________ #
# fig_input_key: str = "display_fn_name:BatchPhoJonathanReplayFRC/plot_result_set:shared/page:1of6/aclus:(2,4,5,6,7,8,9,11,12,13)"
fig_input_key: str = "display_fn_name:BatchPhoJonathanReplayFRC" # plot_result_set:shared/page:1of6/aclus:(2,4,5,6,7,8,9,11,12,13)
# check if the key ends with a forward-slash, in which case it may describe multiple images
# fig_input_key.endswith('/')

_context_figures_dict = neptune_run_collected_results.download_uploaded_figure_files(neptune_project_figures_output_path=neptune_project_figures_output_path, fig_input_key=fig_input_key, debug_print=True)
_context_figures_dict


a_parsed_structure: {'display_fn_name:BatchPhoJonathanReplayFRC': {'plot_result_set:long_only': {'aclus:(13,19,23,29,36,54,60,73,78,90)': <neptune.attributes.atoms.file.File object at 0x00000282B9EC37F0>, 'aclus:(54,60,78,23,90,73,13,36,29,19)': <neptune.attributes.atoms.file.File object at 0x00000282B9EC3640>}, 'plot_result_set:shared': {'page:1of5': {'aclus:(58,41,42,46,55,101,31,63,68,11,69,87,25,91,14,92,2,65,100,86)': <neptune.attributes.atoms.file.File object at 0x00000282B9EC3EE0>}, 'page:1of9': {'aclus:(2,3,5,6,7,9,10,11,12,14)': <neptune.attributes.atoms.file.File object at 0x00000282B9EC3F70>}, 'page:2of5': {'aclus:(59,12,7,74,76,64,103,44,24,34,61,85,38,51,56,9,22,15,66,75)': <neptune.attributes.atoms.file.File object at 0x00000282BE13E1C0>}, 'page:2of9': {'aclus:(15,17,18,20,21,22,24,25,26,28)': <neptune.attributes.atoms.file.File object at 0x00000282BE13E2B0>}, 'page:3of5': {'aclus:(57,37,94,71,95,97,50,17,77,67,18,70,6,79,3,47,82,83,96,26)': <neptune.attributes.atoms.file

Fetching file...: 0 [00:00, ?/s]

	done.
sub_fig_full_input_key: "display_fn_name:BatchPhoJonathanReplayFRC/plot_result_set:shared/page:1of9/aclus:(2,3,5,6,7,9,10,11,12,14)"
a_fig_output_path: "C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\data\neptune\commander.pho\PhoDibaLongShortUpdated\figs\gor01_two_2006-6-08_21-16-25\BatchPhoJonathanReplayFRC-shared-1of9-(2,3,5,6,7,9,10,11,12,14).png"


Fetching file...: 0 [00:00, ?/s]

	done.
sub_fig_full_input_key: "display_fn_name:BatchPhoJonathanReplayFRC/plot_result_set:shared/page:2of5/aclus:(59,12,7,74,76,64,103,44,24,34,61,85,38,51,56,9,22,15,66,75)"
a_fig_output_path: "C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\data\neptune\commander.pho\PhoDibaLongShortUpdated\figs\gor01_two_2006-6-08_21-16-25\BatchPhoJonathanReplayFRC-shared-2of5-(59,12,7,74,76,64,103,44,24,34,61,85,38,51,56,9,22,15,66,75).png"


...

Fetching file...: 0 [00:00, ?/s]

	done.
sub_fig_full_input_key: "display_fn_name:BatchPhoJonathanReplayFRC/plot_result_set:shared/page:4of4/aclus:(38,39)"
a_fig_output_path: "C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\data\neptune\commander.pho\PhoDibaLongShortUpdated\figs\vvp01_one_2006-4-10_12-25-50\BatchPhoJonathanReplayFRC-shared-4of4-(38,39).png"
	done.
sub_fig_full_input_key: "display_fn_name:BatchPhoJonathanReplayFRC/plot_result_set:short_only/aclus:(29,36)"
a_fig_output_path: "C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\data\neptune\commander.pho\PhoDibaLongShortUpdated\figs\vvp01_one_2006-4-10_12-25-50\BatchPhoJonathanReplayFRC-short_only-(29,36).png"
	done.
a_parsed_structure: {'display_fn_name:_display_grid_bin_bounds_validation': <neptune.attributes.atoms.file.File object at 0x00000282BD352CD0>, 'display_fn_name:bidir_track_remap': {'subplot_name:Track Remapping': <neptune.attributes.atoms.file.File object at 0x00000282BD352D30>}, 'display_fn_name:directional_merged_pf_decoded_epochs': {'decoded_epochs:Laps':

Fetching file...: 0 [00:00, ?/s]

	done.
sub_fig_full_input_key: "display_fn_name:BatchPhoJonathanReplayFRC/plot_result_set:shared/page:1of4/aclus:(2,3,4,5,7,8,9,10,11,12)"
a_fig_output_path: "C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\data\neptune\commander.pho\PhoDibaLongShortUpdated\figs\vvp01_one_2006-4-10_12-25-50\BatchPhoJonathanReplayFRC-shared-1of4-(2,3,4,5,7,8,9,10,11,12).png"
	done.
sub_fig_full_input_key: "display_fn_name:BatchPhoJonathanReplayFRC/plot_result_set:shared/page:2of2/aclus:(4,11,7,2,3,14,24,26,28,30,38,39)"
a_fig_output_path: "C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\data\neptune\commander.pho\PhoDibaLongShortUpdated\figs\vvp01_one_2006-4-10_12-25-50\BatchPhoJonathanReplayFRC-shared-2of2-(4,11,7,2,3,14,24,26,28,30,38,39).png"
	done.
sub_fig_full_input_key: "display_fn_name:BatchPhoJonathanReplayFRC/plot_result_set:shared/page:2of4/aclus:(13,14,15,16,17,18,21,22,24,25)"
a_fig_output_path: "C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\data\neptune\commander.pho\PhoDibaLongShortUpdated\figs\vvp01_one_

{Context(format_name= 'kdiba', animal= 'gor01', exper_name= 'two', session_name= '2006-6-08_21-16-25'): {<String field at "sys/id">: {'gor01_two_2006-6-08_21-16-25': {'display_fn_name:BatchPhoJonathanReplayFRC/plot_result_set:long_only/aclus:(13,19,23,29,36,54,60,73,78,90)': 'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/neptune/commander.pho/PhoDibaLongShortUpdated/figs/gor01_two_2006-6-08_21-16-25/BatchPhoJonathanReplayFRC-long_only-(13,19,23,29,36,54,60,73,78,90).png',
    'display_fn_name:BatchPhoJonathanReplayFRC/plot_result_set:long_only/aclus:(54,60,78,23,90,73,13,36,29,19)': 'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/neptune/commander.pho/PhoDibaLongShortUpdated/figs/gor01_two_2006-6-08_21-16-25/BatchPhoJonathanReplayFRC-long_only-(54,60,78,23,90,73,13,36,29,19).png',
    'display_fn_name:BatchPhoJonathanReplayFRC/plot_result_set:shared/page:1of5/aclus:(58,41,42,46,55,101,31,63,68,11,69,87,25,91,14,92,2,65,100,86)': 'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/neptune/

In [ ]:
most_recent_runs_table_df.to_csv('output/2024-09-24_most_recent_neptune_runs_csv.csv')

In [ ]:
most_recent_runs_table_df['final_log_file_path']


In [ ]:
## INPUTS: context_indexed_run_logs, neptune_logs_output_path
## OUTPUT: _out_log_paths
neptune_logs_output_path = neptune_project_output_path.joinpath('logs').resolve()
neptune_logs_output_path.mkdir(exist_ok=True, parents=True)
_out_log_paths = neptune_run_collected_results._perform_export_log_files_to_separate_files(context_indexed_run_logs=context_indexed_run_logs, neptune_logs_output_path=neptune_logs_output_path)
_out_log_paths

# Trivial ipytree Example that causes jumping

In [ ]:
from IPython.display import clear_output, display
from ipytree import Tree, Node


tree = Tree(stripes=False)
for i in range(3):
    node = Node(f"Node {i}")
    tree.add_node(node)
    
def handle_click(event):
    clear_output(wait=True)  # Prevent scrolling
    display(tree)  # Re-display the tree to keep focus
    print(f"Clicked on: {event['owner'].name}")

for node in tree.nodes:
    node.observe(handle_click, names='selected')

tree


# Log Viewer Widget

In [ ]:
# Call the function to build and display the interactive session display
interactive_layout = Neptuner.build_interactive_session_run_logs_widget(context_indexed_run_logs, most_recent_runs_session_descriptor_string_to_context_map, most_recent_runs_context_indexed_run_extra_data)
display(interactive_layout)

In [ ]:
# does not trigger change:
# .on_node_selected(change: {'name': 'selected', 'old': True, 'new': False, 'owner': Node(name='11-02_17-46-44'), 'type': 'change'})
# .on_node_selected(change: {'name': 'selected', 'old': False, 'new': True, 'owner': Node(name='fet11-01_12-58-54', selected=True), 'type': 'change'})
# Selected context: {'format_name': 'kdiba', 'animal': 'pin01', 'exper_name': 'one', 'session_name': 'fet11-01_12-58-54'}


In [ ]:
empty_session_tuple = {
    'format_name': '',
    'animal': '',
    'exper_name': '',
    'session_name': '',
    'session_descriptor_string': '',
    'id': '<Selection Not Found>',
    'hostname': '',
    'creation_time': '',
    'running_time': '',
    'ping_time': '',
    'monitoring_time': '',
    'size': '',
    'tags': '',
    'entrypoint': ''
}

print(list(empty_session_tuple.keys())) # ['format_name', 'animal', 'exper_name', 'session_name', 'session_descriptor_string', 'id', 'hostname', 'creation_time', 'running_time', 'ping_time', 'monitoring_time', 'size', 'tags', 'entrypoint']
display_session_extra_data_keys = ['id', 'hostname', 'creation_time', 'running_time', 'ping_time', 'monitoring_time', 'size', 'tags', 'entrypoint']

# hostname: gl3126.arc-ts.umich.edu
# creation_time: 2024-08-29 16:47:55.150000
# running_time: 8512.876
# ping_time: 2024-09-24 08:38:06.349000
# monitoring_time: 1291
# size: 24744223.0
# tags: one,kdiba,11-02_19-28-0,pin01
# entrypoint: figures_kdiba_pin01_one_11-02_19-28-0.py



In [ ]:
from pandas import Timestamp
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.gui.Jupyter.TreeWidget import JupyterTreeWidget

## INPUTS: context_indexed_run_logs, most_recent_runs_session_descriptor_string_to_context_map, header_hbox

# Tree Widget ________________________________________________________________________________________________________ #
included_session_contexts: List[IdentifyingContext] = list(most_recent_runs_session_descriptor_string_to_context_map.values())
jupyter_tree_widget = JupyterTreeWidget(included_session_contexts=included_session_contexts,
										on_selection_changed_callbacks=[],
										display_on_init=False)
# type(jupyter_tree_widget.tree) # ipytree.tree.Tree

# Set a layout for the tree to prevent it from being cut off
jupyter_tree_widget.tree.layout = widgets.Layout(min_width='200px', max_width='600px', overflow='auto', height='auto')

# Content Widget _____________________________________________________________________________________________________ #
def build_session_tuple_header_widget(a_session_tuple: Tuple):
    # Create widgets for each key-value pair
    # widgets_list = [widgets.Label(f"{key}: {value}") for key, value in a_session_tuple.items()]
    
    # Create a dictionary to hold the label widgets
    header_label_widgets = {key: widgets.Label(f"{key}: '{value}',") for key, value in a_session_tuple.items()}

    # Horizontally stack the label widgets
    # header_hbox = widgets.HBox(list(header_label_widgets.values()), layout=widgets.Layout(min_width='400px', min_height='50px', width='auto', height='auto')) # , overflow='auto'
    
    # Define a layout that enables wrapping
    box_layout = widgets.Layout(
        display='flex',
        flex_flow='row wrap',
        align_items='stretch',
        width='100%'
    )

    # Create a Box with the custom layout
    header_hbox = widgets.Box(list(header_label_widgets.values()), layout=box_layout)


    # Function to update the values in the labels
    def update_header_labels_fn(new_values):
        """ captures: label_widgets"""
        for key, value in new_values.items():
            header_label_widgets[key].value = f"{key}: {value}"
            
    # Display the widget
    return header_hbox, header_label_widgets, update_header_labels_fn


# Example SessionTuple
empty_session_tuple = {
    'format_name': '',
    'animal': '',
    'exper_name': '',
    'session_name': '',
    'session_descriptor_string': '',
    'id': '<Selection Not Found>',
    'hostname': '',
    'creation_time': '',
    'running_time': '',
    'ping_time': '',
    'monitoring_time': '',
    'size': '',
    'tags': '',
    'entrypoint': ''
}

header_hbox, header_label_widgets, update_header_labels_fn = build_session_tuple_header_widget(a_session_tuple=empty_session_tuple)

# Create Textarea widget with a defined width
textarea = widgets.Textarea(value='<No Selection>',
    disabled=True,  # Make it read-only
    style={'font_size': '10px'},  # Smaller font size
	layout=widgets.Layout(flex='1', width='650px', min_height='650px',
                        height='850px',
                        ))

content_view_layout = widgets.VBox([header_hbox, textarea], layout=widgets.Layout(min_width='500px', min_height='200px', width='auto', height='auto')) # INPUTS: header_hbox

# Layout widgets side by side with proper spacing
layout = widgets.HBox([jupyter_tree_widget.tree, content_view_layout], layout=widgets.Layout(min_width='500px', min_height='100px', width='auto', height='auto'))


def _on_tree_node_selection_changed(selected_node, selected_context):
    """ 
    captures: context_indexed_run_logs, textarea, most_recent_runs_context_indexed_run_extra_data, empty_session_tuple, update_header_labels_fn
    """
    print(f'_on_tree_node_selection_changed(selected_node: {selected_node}, selected_context: {selected_context})') # Selected context: {'format_name': 'kdiba', 'animal': 'pin01', 'exper_name': 'one', 'session_name': 'fet11-01_12-58-54'}
    #_on_tree_node_selection_changed(selected_node: Node(name='fet11-01_12-58-54', selected=True), selected_context: {'format_name': 'kdiba', 'animal': 'pin01', 'exper_name': 'one', 'session_name': 'fet11-01_12-58-54'})
    if isinstance(selected_context, dict):
        selected_context = IdentifyingContext(**selected_context)
    
    curr_context_extra_data_tuple = most_recent_runs_context_indexed_run_extra_data.get(selected_context, empty_session_tuple)
    update_header_labels_fn(curr_context_extra_data_tuple)
    curr_context_run_log: str = context_indexed_run_logs.get(selected_context, '<Context Not Found>')
    textarea.value = curr_context_run_log

jupyter_tree_widget.on_selection_changed_callback = [_on_tree_node_selection_changed]
# Display the layout
layout


In [ ]:


text_area = widgets.Textarea(
    value='<No Selection>',
    disabled=True,  # Make it read-only
    layout=widgets.Layout(width='100%', height='400px')  # Scrollable area
)

combined_widget = widgets.HBox([jupyter_tree_widget.tree, text_area])
combined_widget
# log_viewer = create_log_viewer(logs=run_logs)
# display(log_viewer)


In [ ]:
parsed_structure = run.get_parsed_structure()
parsed_structure

In [ ]:
import regions
(parsed_structure['monitoring'])


In [ ]:
for run_id, a_log in run_logs.items():
    print("# ==================================================================================================================== #")
    print(f"# run_id: {run_id}                                                                                                     #")
    print("# ==================================================================================================================== #")    
    # print(f'run_id: {run_id} =================')
    # print(f"# run_id: {run_id:<100} #")
    print(a_log)
    print("# END LOG ____________________________________________________________________________________________________________ #")
    


Calling `run.print_structure()` produces the following output:
```python
'animal': String
'exper_name': String
'format_name': String
'monitoring':
    '5f739afe':
        'hostname': String
        'pid': String
        'tid': String
    'be28f54f':
        'cpu': FloatSeries
        'hostname': String
        'memory': FloatSeries
        'pid': String
        'stderr': StringSeries
        'stdout': StringSeries
        'tid': String
'outputs':
    'figures':
        'display_fn_name:BatchPhoJonathanReplayFRC':
            'plot_result_set:long_only':
                'aclus:(16,19,6,31,10)': File
                'aclus:(6,10,16,19,31)': File
            'plot_result_set:shared':
                'page:1of2':
                    'aclus:(26,20,29,18,3,4,7,32,13,11,12,2,30,15,9,28,22,14,21,25)': File
                'page:1of3':
                    'aclus:(2,3,4,5,7,8,9,11,12,13)': File
                'page:2of2':
                    'aclus:(5,8,17,23,27)': File
                'page:2of3':
                    'aclus:(14,15,17,18,20,21,22,23,25,26)': File
                'page:3of3':
                    'aclus:(27,28,29,30,32)': File
            'plot_result_set:short_only':
                'aclus:(24)': File
        'display_fn_name:_display_grid_bin_bounds_validation': File
        'display_fn_name:bidir_track_remap':
            'subplot_name:Track Remapping': File
        'display_fn_name:display_long_short_laps': File
        'display_fn_name:display_short_long_pf1D_comparison':
            'track:long': File
            'track:short': File
        'display_fn_name:plot_all_epoch_bins_marginal_predictions':
            'subplot_name:Laps all_epoch_binned Marginals': File
            'subplot_name:Ripple all_epoch_binned Marginals': File
        'display_fn_name:plot_expected_vs_observed':
            'x_variable:epoch_idx':
                'variable:obs_exp_diff_ptp': File
        'display_fn_name:plot_histograms':
            'subplot_name:laps_result_tuple': File
            'subplot_name:ripple_result_tuple': File
        'display_fn_name:plot_quantile_diffs':
            'subplot_name:BestDir': File
        'display_fn_name:plot_rank_order_epoch_inst_fr_result_tuples':
            'subplot_name:Lap':
                'subplotsubplot_name:raw': File
                'subplotsubplot_name:z_score_diff': File
            'subplot_name:Ripple':
                'subplotsubplot_name:raw': File
                'subplotsubplot_name:z_score_diff': File
        'display_fn_name:plot_rank_order_histograms':
            'subplot_name:Ripple Most-likely Spearman Rho': File
            'subplot_name:Ripple Most-likely Z-scores': File
            'subplot_name:Ripple Z-scores': File
            'subplot_name:Ripple real correlations': File
        'display_fn_name:running_and_replay_speeds_over_time': File
        'filter_name:maze1_any':
            'lap_dir:any':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
                'display_fn_name:plot_single_track_firing_rate_compare': File
        'filter_name:maze1_even':
            'lap_dir:even':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze1_odd':
            'lap_dir:odd':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze2_any':
            'lap_dir:any':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
                'display_fn_name:plot_single_track_firing_rate_compare': File
        'filter_name:maze2_even':
            'lap_dir:even':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze2_odd':
            'lap_dir:odd':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze_any':
            'lap_dir:any':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze_even':
            'lap_dir:even':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze_odd':
            'lap_dir:odd':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'fn_name:long_short_firing_rate_indicies':
            'display_fn_name:display_long_short_laps': File
'session_descriptor_string': String
'session_name': String
'source_code':
    'diff': File
    'entrypoint': String
'sys':
    'creation_time': Datetime
    'description': String
    'failed': Boolean
    'group_tags': StringSet
    'hostname': String
    'id': String
    'modification_time': Datetime
    'monitoring_time': Integer
    'name': String
    'owner': String
    'ping_time': Datetime
    'running_time': Float
    'size': Float
    'state': RunState
    'tags': StringSet
    'trashed': Boolean
```
How can I capture this printed output (printed to stdout I think) and then parse it into a tree-like structure made of nested-dictionaries to be able to figure out the available structure?

In [ ]:
# Fetch all experiments
experiments = neptuner.project.fetch_experiments_table().to_pandas()
experiments

In [ ]:
# Initialize the neptune client
# neptune.init(project=neptune_kwargs['project'], api_token=neptune_kwargs['api_token'])

# project = neptune.get_project()
# project

In [ ]:


if neptuner.run is None:
    neptuner.run = AutoValueConvertingNeptuneRun(project=neptuner.project_name, api_token=neptuner.api_token, dependencies="infer", source_files=[notebook_filepath])
    params = {"TODAY_DAY_DATE": TODAY_DAY_DATE, "run_workstation": "Apogee"}
    neptuner.run["parameters"] = params
    neptuner.outputs = neptuner.run['outputs']
    neptuner.figures = neptuner.outputs['figures']

neptuner_run: AutoValueConvertingNeptuneRun = neptuner.run

# run = neptune.init_run(source_files=["**/*.dvc"])

# # Pre-execution dataframe view:
# run["dataset/global_batch_run_progress_df"].upload(File.as_html(global_batch_run.to_dataframe(expand_context=True, good_only=False))) # "path/to/test_preds.csv"



known_bad_sessions = [IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44')]
known_bad_session_strs = [str(v.get_description()) for v in known_bad_sessions]
known_bad_session_strs